In [1]:
import requests
import json
import pandas as pd

In [2]:
key = '6cdee3cbd6d29e49fdc6bd17a2feb85b'

In [3]:
day = '20220922'

In [4]:
url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={}&targetDt={}'.format(key, day)

In [5]:
result = requests.get(url)
text = result.text
d = json.loads(text)

li = []
li2 = ['rank', 'movieCd', 'movieNm', 'openDt', 'audiCnt', 'audiAcc', 'salesAmt', 'salesAcc', 'scrnCnt', 'showCnt']
for b in d['boxOfficeResult']['dailyBoxOfficeList']:
    li.append([b[li2[i]] for i in range(0, len(li2))])

data = pd.DataFrame(li)
data.columns = ['순위', '영화코드', '제목', '개봉일', '당일관객수', '누적관객수', '당일매출액', '누적매출액', '당일스크린수', '당일상영횟수']
data

,순위,영화코드,제목,개봉일,당일관객수,누적관객수,당일매출액,누적매출액,당일스크린수,당일상영횟수
0,1,20215601,공조2: 인터내셔날,2022-09-07,73488,5070973,718995735,52194834477,1568,6421
1,2,20211472,늑대사냥,2022-09-21,48139,129860,480962576,1288274350,1146,4718
2,3,20226957,아바타 리마스터링,2022-09-21,9771,24078,157982570,388731100,281,1019
3,4,20217826,육사오(6/45),2022-08-24,9414,1879054,79352602,18931338235,561,1111
4,5,20226751,극장판 5등분의 신부,2022-09-22,6360,6360,69176000,69176000,89,222
5,6,20198317,인생은 아름다워,2022-09-28,3679,20300,34057000,202032595,22,22
6,7,20219628,정직한 후보2,2022-09-28,2593,8423,22337000,74807000,15,15
7,8,20226568,애프터: 에버 해피,2022-09-21,1781,3159,12541400,25519400,124,273
8,9,20211792,헌트,2022-08-10,1550,4338150,11379793,44475219256,194,270
9,10,20196776,프린세스 아야,2022-09-21,1402,4065,9057200,33001099,206,250


In [9]:
movie_list = []
for i in range(len(data)):
    movieCd = data['영화코드'][i]
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={}&movieCd={}'.format(key, movieCd)
    
    result = requests.get(url)
    text = result.text
    d_json = json.loads(text)

    li = []
    li2 = ['movieCd', 'movieNm', 'nations', 'directors', 'genres']

    for i in range(0, len(li2)):
        if type(d_json['movieInfoResult']['movieInfo'][li2[i]]) == list:
            sub_li = []
            for j in range(0, len(d_json['movieInfoResult']['movieInfo'][li2[i]])):
                sub_li.append(list(d_json['movieInfoResult']['movieInfo'][li2[i]][j].values())[0])

            str = ", ".join(sub_li)
            li.append(str)
        else :
            li.append(d_json['movieInfoResult']['movieInfo'][li2[i]])
            
    movie_list.append(li)
    
movie_detail = pd.DataFrame(movie_list, columns = li2)

In [10]:
movie_detail

,movieCd,movieNm,nations,directors,genres
0,20215601,공조2: 인터내셔날,한국,이석훈,"액션, 코미디"
1,20211472,늑대사냥,한국,김홍선,"액션, 범죄"
2,20226957,아바타 리마스터링,미국,제임스 카메론,"SF, 어드벤처, 액션, 전쟁"
3,20217826,육사오(6/45),한국,박규태,코미디
4,20226751,극장판 5등분의 신부,일본,진보 마사토,"애니메이션, 드라마"
5,20198317,인생은 아름다워,한국,최국희,뮤지컬
6,20219628,정직한 후보2,한국,장유정,코미디
7,20226568,애프터: 에버 해피,미국,캐스틸 랜던,멜로/로맨스
8,20211792,헌트,한국,이정재,"액션, 드라마"
9,20196776,프린세스 아야,한국,이성강,"판타지, 뮤지컬"
